In [5]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

from keras.datasets import mnist
#import matplotlib
#%matplotlib inline
#import matplotlib.pyplot as plt
#import l2_attack
import keras
#from defense import *
import tensorflow as tf
import time
import torch
import numpy as np

from wrapper import Model

In [6]:
class blackbox:
    def __init__(self,model):
        self.model = model
        
    def attack_untargeted(self, x0, y0, alpha = 2, beta = 0.005, iterations = 1000):
        """ Attack the original image and return adversarial example
            model: (pytorch model)
            alpha: learning rate 
            beta: learning rate
            train_dataset: set of training data
            (x0, y0): original image
        """
        
        if (self.model.predict(x0)!= y0):
            print("Fail to classify the image. No need to attack.")
            return x0,0
    
        num_directions = 1000
        best_theta, g_theta = None, float('inf')
        query_count = 0
        
        timestart = time.time()
        for i in range(num_directions):
            theta = torch.randn(x0.shape).type(torch.FloatTensor)
            initial_lbd = torch.norm(theta)
            theta = theta/torch.norm(theta)
            if self.model.predict(x0+np.array(initial_lbd*theta)) != y0:
                lbd, count = self.fine_grained_binary_search( x0, y0, theta, initial_lbd, g_theta)
                query_count += count
                if lbd < g_theta:
                    best_theta, g_theta = theta,lbd
                    print("--------> Found distortion %.4f" % g_theta)
        timeend = time.time()
        print("==========> Found best distortion %.4f in %.4f seconds using %d queries" % (g_theta, timeend-timestart, query_count))
        
           
            
        
        #timestart = time.time()
        #print("the best initialization: ",best_theta)
        g1 = 1.0
        theta, g2 = best_theta.clone(), g_theta
        torch.manual_seed(0)
        opt_count = 0
        stopping = 0.01
        prev_obj = 100000
        
        for i in range(iterations):
            if g_theta < 1:
                print("====================query number after distortion < 1 =======================: ",opt_count)
                break
            #print("n_query:",opt_count)
            #print("best distortion:", g_theta)
#            print("iteration:", i )
            gradient = torch.zeros(theta.size())
            q = 10
            min_g1 = float('inf')
            for _ in range(q):
                u = torch.randn(theta.size()).type(torch.FloatTensor)
                u = u/torch.norm(u)
                ttt = theta+beta * u
                ttt = ttt/torch.norm(ttt)
                g1, count = self.fine_grained_binary_search_local( x0, y0, ttt, initial_lbd = g2, tol=beta/500)
                opt_count += count
                gradient +=  (g1-g2)/beta * u
                #print("norm of gradient:", np.linalg.norm(gradient))
                if g1 < min_g1:
                    min_g1 = g1
                    min_ttt = ttt
            gradient = 1.0/q * gradient
    
            if (i+1)%50 == 0:
                
                print("Iteration %3d: g(theta + beta*u) = %.4f g(theta) = %.4f distortion %.4f num_queries %d" % (i+1, g1, g2, torch.norm(g2*theta), opt_count))
                if g2 > prev_obj-stopping:
                    break
                prev_obj = g2
    
            min_theta = theta
            min_g2 = g2
        
            for _ in range(15):
                #print("enter first for loop")
                new_theta = theta - alpha * gradient
                new_theta = new_theta/torch.norm(new_theta)
                new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                opt_count += count
                alpha = alpha * 2
                #print("alpha in the first for loop is: ",alpha)
                if new_g2 < min_g2:
                    min_theta = new_theta 
                    min_g2 = new_g2
                else:
                    break
    
            if min_g2 >= g2:
                for _ in range(15):
                    #print("enter second for loop")
                    alpha = alpha * 0.8
                    new_theta = theta - alpha * gradient
                    new_theta = new_theta/torch.norm(new_theta)
                    new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                    opt_count += count
                    #print("alpha in the second for loop is: ",alpha)
                    if new_g2 < g2:
                        min_theta = new_theta 
                        min_g2 = new_g2
                        break
    
            if min_g2 <= min_g1:
                theta, g2 = min_theta, min_g2
            else:
                theta, g2 = min_ttt, min_g1
    
            if g2 < g_theta:
                best_theta, g_theta = theta.clone(), g2
            
            #print(alpha)
#            print("%3d th iteration" % i)
            #print("current alpha:",alpha)
            if alpha < 1e-4:
                alpha = 1.0
                #print("Warning: not moving, g2 %lf gtheta %lf" % (g2, g_theta))
                beta = beta * 0.1
                if (beta < 0.0005):
                    break
    
        #target = model.predict(x0 + g_theta*best_theta)
        
        #timeend = time.time()
        #print("\nAdversarial Example Found Successfully: distortion %.4f target %d queries %d \nTime: %.4f seconds" % (g_theta, target, query_count + opt_count, timeend-timestart))
        print("mnist baseline 2")
        print("best distortion :", g_theta)
        print("number of queries :", opt_count+query_count )
        return np.array(g_theta*best_theta), opt_count+query_count 
    def fine_grained_binary_search_local(self, x0, y0, theta, initial_lbd = 1.0, tol=1e-5):
        nquery = 0
        lbd = initial_lbd
         
        if self.model.predict(x0+np.array(lbd*theta)) == y0:
            lbd_lo = lbd
            lbd_hi = lbd*1.01
            nquery += 1
            while self.model.predict(x0+np.array(lbd_hi*theta)) == y0:
                lbd_hi = lbd_hi*1.01
                nquery += 1
                if lbd_hi > 20:
                    return float('inf'), nquery
        else:
            lbd_hi = lbd
            lbd_lo = lbd*0.99
            nquery += 1
            while self.model.predict(x0+np.array(lbd_lo*theta)) != y0 :
                lbd_lo = lbd_lo*0.99
                nquery += 1
    
        while (lbd_hi - lbd_lo) > tol:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        return lbd_hi, nquery
    
    def fine_grained_binary_search(self, x0, y0, theta, initial_lbd, current_best):
        nquery = 0
        if initial_lbd > current_best: 
            if self.model.predict(x0+ np.array(current_best*theta)) == y0:
                nquery += 1
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd
        
        ## original version
        #lbd = initial_lbd
        #while model.predict(x0 + lbd*theta) == y0:
        #    lbd *= 2
        #    nquery += 1
        #    if lbd > 100:
        #        return float('inf'), nquery
        
        #num_intervals = 100
    
        # lambdas = np.linspace(0.0, lbd, num_intervals)[1:]
        # lbd_hi = lbd
        # lbd_hi_index = 0
        # for i, lbd in enumerate(lambdas):
        #     nquery += 1
        #     if model.predict(x0 + lbd*theta) != y0:
        #         lbd_hi = lbd
        #         lbd_hi_index = i
        #         break
    
        # lbd_lo = lambdas[lbd_hi_index - 1]
        lbd_hi = lbd
        lbd_lo = 0.0
    
        while (lbd_hi - lbd_lo) > 1e-5:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        return lbd_hi, nquery
    

In [ ]:
session = keras.backend.get_session()
keras.backend.set_learning_phase(False)
model = keras.models.load_model("data/mnist")
model = Model(model,[0.0,1.0])

attack = blackbox(model)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test = np.array(x_test, dtype=np.float32)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test /= 255.0

count = []
for i in range(20):
    label = model.predict(x_test[i])
    if label == y_test[i]:
        count.append(1)
    else:
        count.append(0)
    
print("accuracy of this model is:", sum(count)/len(count))


accuracy of this model is: 0.95


In [ ]:
dist = []
count = []
for i in range(15):
    print("=========================image ",i+1,"==========================================")
    print("true label:",y_test[i+20])
    print("predicted label:",model.predict(x_test[i+20]))
    adv, queries= attack.attack_untargeted(x_test[i+20],y_test[i+20], alpha = 4, beta = 0.005, iterations = 1000)
    res = adv - x_test[i+20]
    dist.append(np.linalg.norm(res))
    count.append(queries)

index = np.nonzero(count)
index = list(index)[0].tolist()

avg_distortion = np.mean(np.array(dist)[index])
avg_count = np.mean(np.array(count)[index])
print("the average distortion for %2d images :"%(len(index)),avg_distortion)
for i in dist:
    print(i)
print("the number of queries for %2d images :"%(len(index)), avg_count)
for j in count:
    print(j)

=========================image  1 ==========================================
true label: 9
predicted label: 9
--------> Found distortion 11.2897
--------> Found distortion 9.4779
--------> Found distortion 9.4475
--------> Found distortion 9.0865
--------> Found distortion 8.7925
--------> Found distortion 8.4800
--------> Found distortion 6.3860
==========> Found best distortion 6.3860 in 4.3521 seconds using 1130 queries
Iteration  50: g(theta + beta*u) = 1.7780 g(theta) = 1.7794 distortion 1.7794 num_queries 16884
Iteration 100: g(theta + beta*u) = 1.3446 g(theta) = 1.3447 distortion 1.3447 num_queries 31348
Iteration 150: g(theta + beta*u) = 1.1818 g(theta) = 1.1818 distortion 1.1818 num_queries 43921
Iteration 200: g(theta + beta*u) = 1.0936 g(theta) = 1.0936 distortion 1.0936 num_queries 55379
Iteration 250: g(theta + beta*u) = 1.0333 g(theta) = 1.0332 distortion 1.0332 num_queries 66154
====================query number after distortion < 1 =======================:  73755
mnist b

Iteration 650: g(theta + beta*u) = 1.5064 g(theta) = 1.5064 distortion 1.5064 num_queries 147201
mnist baseline 2
best distortion : tensor(1.5064)
number of queries : 148321
=========================image  6 ==========================================
true label: 0
predicted label: 0
--------> Found distortion 22.1770
--------> Found distortion 17.8939
--------> Found distortion 16.1658
--------> Found distortion 15.4228
--------> Found distortion 15.3293
--------> Found distortion 13.6165
--------> Found distortion 12.7040
--------> Found distortion 11.1775
--------> Found distortion 10.8304
==========> Found best distortion 10.8304 in 3.8383 seconds using 1141 queries
Iteration  50: g(theta + beta*u) = 3.9726 g(theta) = 3.9712 distortion 3.9712 num_queries 16285
Iteration 100: g(theta + beta*u) = 3.0883 g(theta) = 3.0882 distortion 3.0882 num_queries 30802
Iteration 150: g(theta + beta*u) = 2.6740 g(theta) = 2.6743 distortion 2.6743 num_queries 44176
Iteration 200: g(theta + beta*u) =